### Testing the Hessian as a derivative of certainty 

In [1]:
options(jupyter.plot_mimetypes = 'image/png')
set.seed(123)
library(TMB)

Loading required package: Matrix


$$
Y \sim \mathcal{N}(\mu, \sigma)
$$

In [2]:
N1 <- 10000
N2 <- 10
y1 <- rnorm(N1, 5, 1)
y2 <- rnorm(N2, 5, 1)

In [3]:
nll_func_simple <- function(par, data){
    mu <- par[1]
    sd_ <- par[2]
    -1 * sum(dnorm(data, mean=mu, sd=sd_, log=TRUE))
} 

lm_opt1 <- optim(par=c(1,1), fn=nll_func_simple, data=y1, lower=c(-Inf, 0.0001), 
                 upper=Inf, method="L-BFGS-B", hessian=TRUE)
lm_opt2 <- optim(par=c(1,1), fn=nll_func_simple, data=y2, lower=c(-Inf, 0.0001), 
                 upper=Inf, method="L-BFGS-B", hessian=TRUE)

In [4]:
print(rbind(lm_opt1$par, lm_opt2$par))

         [,1]      [,2]
[1,] 4.997629 0.9985866
[2,] 5.316846 1.0848680


In [5]:
print(sqrt(diag(solve(lm_opt1$hessian))))

[1] 0.009985866 0.007061040


In [6]:
print(sqrt(diag(solve(lm_opt2$hessian))))

[1] 0.3430654 0.2425832


In [7]:
fileConn<-file("lm_test.cpp")

cpp_code <- 
'
#include <TMB.hpp>
template<class Type>
Type objective_function<Type>::operator() ()
{
  // Data
  DATA_VECTOR(y);

  // Parameters
  PARAMETER(mean);
  PARAMETER(log_sd);

  // Objective funcction
  Type sd = exp(log_sd);

  // Probability of data conditional on fixed effect values
  Type jnll = -1 * sum(dnorm(y, mean, sd, true));
  
  // Reporting
  return jnll;
}
'

writeLines(cpp_code, fileConn)
close(fileConn)

In [8]:
if(!(dynlib("lm_test") %in% list.files())){
    compile("lm_test.cpp")
}

In [9]:
# Step 2 -- build inputs and object
dyn.load(dynlib("lm_test"))
Params <- list(mean=0, log_sd=0)
Data <- list(y=y1)
Obj <- MakeADFun( data=Data, parameters=Params, DLL="lm_test")

# Step 3 -- test and optimize
Obj$fn(Obj$par)
Obj$gr(Obj$par)
Opt <- nlminb(start=Obj$par, objective=Obj$fn, gradient=Obj$gr)
Opt$diagnostics = data.frame(name=names(Obj$par), Est=Opt$par, 
                             final_gradient=as.vector(Obj$gr(Opt$par)))
SD <- sdreport(Obj) # standard errors
SD

[1] 139056.7

outer mgc:  249734.6 


-49976.28,-249734.6


outer mgc:  249734.6 
outer mgc:  23839.52 
outer mgc:  4373.277 
outer mgc:  3047.535 
outer mgc:  1134.511 
outer mgc:  497.9886 
outer mgc:  208.9221 
outer mgc:  43.2162 
outer mgc:  6.986684 
outer mgc:  0.7983885 
outer mgc:  0.0007113673 
outer mgc:  1.573886e-06 
outer mgc:  1.573886e-06 
outer mgc:  1.573886e-06 
outer mgc:  10.02833 
outer mgc:  10.02833 
outer mgc:  19.98001 
outer mgc:  20.02001 


In [11]:
Params <- list(mean=0, log_sd=0)
Data <- list(y=y2)
Obj <- MakeADFun( data=Data, parameters=Params, DLL="lm_test")

# Step 3 -- test and optimize
Obj$fn(Obj$par)
Obj$gr(Obj$par)
Opt <- nlminb(start=Obj$par, objective=Obj$fn, gradient=Obj$gr)
Opt$diagnostics <- data.frame(name=names(Obj$par), Est=Opt$par, 
                             final_gradient=as.vector(Obj$gr(Opt$par)))
SD <- sdreport(Obj) # standard errors
SD

[1] 156.4184

outer mgc:  284.4579 


-53.16846,-284.4579


outer mgc:  284.4579 
outer mgc:  28.54229 
outer mgc:  21.81578 
outer mgc:  4.656971 
outer mgc:  4.397002 
outer mgc:  3.671862 
outer mgc:  2.854891 
outer mgc:  0.9646584 
outer mgc:  0.2967955 
outer mgc:  0.0326299 
outer mgc:  0.000445752 
outer mgc:  5.382956e-07 
outer mgc:  4.649827e-10 
outer mgc:  4.649827e-10 
outer mgc:  4.649827e-10 
outer mgc:  0.008496633 
outer mgc:  0.008496632 
outer mgc:  0.01998001 
outer mgc:  0.02002001 


         Estimate Std. Error
mean   5.31684645  0.3430651
log_sd 0.08145761  0.2236067

Maximum gradient component: 4.649827e-10 